In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import glob
import yaml
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from models.ae_modules import *

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle
from sklearn.datasets import fetch_olivetti_faces
from data.create_dataset import create_dataloader, create_dataloader_18,create_dataloader_only12
#from skimage.transform import resize


type_ae="VAE"
net ="Normal" #ResNet(ResNet18), ResNet18s(channel with s=1), Normal
s=1 # ResNetのチャネル数 2 : {64, 128, 256, 512} 1 : {32, 64, 128, 256}

epoch = 800
CNN_embed_dim = 64    # latent dim extracted by 2D CNN
batch_size = 64
input_size = (100,192) 
batch_size=1

# save model
num_lead = 12
task = 5
seed=50
times=2
io = 'in_asc'

saved_model_path ='./runs/view{}_{}_{}/{}/{}/{}'.format(num_lead,task,io,type_ae,net,times)
input_channel=3 

use_cuda = torch.cuda.is_available()                   # check if GPU exists
device = torch.device("cuda" if use_cuda else "cpu")   # use CPU or GPU
print(device)
# reload ResNetVAE model
# Create model
if type_ae == "AE":
    if net == "Normal":
        model = AE(batch_size=batch_size,input_ch=input_channel,CNN_embed_dim=CNN_embed_dim, input_size=input_size).to(device)
    else:
        model = ResNet_AE(batch_size=batch_size,input_ch=input_channel,CNN_embed_dim=CNN_embed_dim,input_size=input_size,s=s).to(device)
    model.load_state_dict(torch.load(os.path.join(saved_model_path+'/model/', 'model_epoch{}.pth'.format(epoch)),map_location=torch.device(device)))
elif type_ae == "VAE":
    if net == "Normal":
        model = VAE(batch_size=batch_size,input_ch=input_channel,CNN_embed_dim=CNN_embed_dim, input_size=input_size).to(device)
    else:
        model = ResNet_VAE(batch_size=batch_size,input_ch=input_channel,CNN_embed_dim=CNN_embed_dim,input_size=input_size,s=s).to(device)
    model.load_state_dict(torch.load(os.path.join(saved_model_path+'/model/', 'model_epoch{}.pth'.format(epoch)),map_location=torch.device(device)))

print('epoch {} model reloaded!'.format(epoch))

da = True
if num_lead == 12:
    if task == 2:
        data_dir = './dataset/ECG100_224_2class/'
        train_dataloader, test_dataloader, label_name, train_len, test_len = create_dataloader_only12(data_dir, batch_size,input_size,seed,io,da)
        print('Data Augumentation : ', da)

    elif task == 4:
        data_dir = './dataset/ECG100_224_ACS/'
        train_dataloader, test_dataloader, label_name, train_len, test_len = create_dataloader_only12(data_dir, batch_size,input_size,seed,io,da)
        print('Data Augumentation : ', da)

    elif task == 5:
        data_dir = './dataset/ECG100_224/'
        train_dataloader, test_dataloader, label_name, train_len, test_len = create_dataloader_only12(data_dir, batch_size,input_size,seed,io,da)
        print('Data Augumentation : ', da)
                
elif num_lead == 18:
    if task == 2:
        data_dir = './dataset/ECG100_224_2class/'
        train_dataloader, test_dataloader, label_name, train_len, test_len = create_dataloader_18(data_dir, batch_size,input_size,seed,io,da)
        print('Data Augumentation : ', da)
            
    elif task == 4:
        data_dir = './dataset/ECG100_224_ACS/'
        train_dataloader, test_dataloader, label_name, train_len, test_len = create_dataloader_18(data_dir, batch_size,input_size,seed,io,da)

    elif task == 5:
        data_dir = './dataset/ECG100_224/'
        train_dataloader, test_dataloader, label_name, train_len, test_len = create_dataloader_18(data_dir, batch_size,input_size,seed,io,da)
        print('Data Augumentation : ', da)
    
elif num_lead == 19:
    if task == 2:
        data_dir = './dataset/ECG100_224_2class/'
        train_dataloader, test_dataloader, label_name, train_len, test_len = create_dataloader(data_dir, batch_size,input_size,seed,io,da)
        print('Data Augumentation : ', da)

    elif task == 4:
        data_dir = './dataset/ECG100_224_ACS/'
        train_dataloader, test_dataloader, label_name, train_len, test_len = create_dataloader(data_dir, batch_size,input_size,seed,io,da)
        print('Data Augumentation : ', da)
            
    elif task == 5:
        data_dir = './dataset/ECG100_224/'
        train_dataloader, test_dataloader, label_name, train_len, test_len = create_dataloader(data_dir, batch_size,input_size,seed,io,da)
        print('Data Augumentation : ', da)


/python/build/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


FileNotFoundError: [Errno 2] No such file or directory: './runs/view12_5_in_asc/VAE/Normal/2/model/model_epoch800.pth'

In [ ]:
x = torch.zeros(1,3,100,192,dtype=torch.float32).to(device)
model.eval()
model(x)
a=1

In [ ]:
train_loss = np.load(os.path.join(saved_model_path, 'train_loss.npy'))
test_loss = np.load(os.path.join(saved_model_path, 'test_loss.npy'))
print(train_loss.shape)
print(test_loss.shape)
ep = np.arange(train_loss.shape[0])
plt.figure(figsize=(8,6))
plt.plot(ep,train_loss,label='train')
plt.plot(ep,test_loss,label='test')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.ylim(0,10000)
plt.legend()
plt.savefig(saved_model_path+"/loss.png", bbox_inches='tight', dpi=600)
plt.show()
    

## Reconstruction 

In [ ]:
model.eval()
reconstruction_path = saved_model_path+'/reconstruction'
if not os.path.exists(reconstruction_path):
    os.mkdir(reconstruction_path)

print('train_data')
for s in range(3):
    N=5
    pos=1
    fig = plt.figure(figsize=(6, 10))
    for i, (data,y,_,_) in enumerate(train_dataloader):
        data = data.to(device)

        x = data[:,0,:,:,:]
        if type_ae == 'VAE':
            data_recontruction, z, mu, logvar = model(x)  # VAE
        else:
            data_recontruction, z = model(x)
        #data_recontruction = model(data)[0]
        data_ = data[0][0].squeeze(0).data.cpu().numpy().transpose((1,2,0))
        print(data_recontruction[0].size())
        data_recontruction_ = data_recontruction[0].squeeze_(0).data.cpu().numpy().transpose((1,2,0))

        plt.subplot(N, 2, pos)
        plt.imshow(data_)
        plt.gray()
        plt.title('original')
        plt.axis('off')
        pos +=1

        plt.subplot(N, 2, pos)
        plt.imshow(data_recontruction_)
        plt.gray()
        plt.title('reconstructed')
        plt.axis('off')
        pos +=1

        if i == 2:
            break
    plt.savefig(reconstruction_path+"/train_reconst_sample{}.png".format(s), bbox_inches='tight', dpi=600)
    plt.show()
